In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [2]:
import argiope, compmod2
import pandas as pd
import numpy as np
from string import Template

In [3]:
def _control_average_value(free_nodes, control_node, dof, **kwargs):
        """
        Fixes the averave value of free node's displacement along a given dof.
        """
        return argiope.utils._equation(nodes = np.concatenate([free_nodes, [control_node]]), 
                                       dofs = np.ones(len(free_nodes) + 1) * dof,
                                       coefficients = np.concatenate([np.ones(len(free_nodes)),
                                                                     [-len(free_nodes)]]), **kwargs)

class CuboidSample(argiope.models.Part):
    """
    A cuboid test sample.
    """
    def __init__(self, shape = (1, 1, 1), dim = (1., 1., 1.)):
        self.dim = dim
        self.shape = shape

    def make_mesh(self):
        """
        Makes the mesh
        """    
        mesh = argiope.mesh.structured_mesh(dim = self.dim, shape = self.shape)
        self.mesh = mesh

In [4]:
'''
def get_node_pairs(pair, normal = "x"):
    """
    Returns pairs of nodes that are in the same position on two faces modulo
    a translation along a normal direction.
    """
    plane = ["x", "y", "z"]
    plane = [d for d in plane if d != normal]
    pair = [left, right]
    pair = [d.coords[plane].sort_values(plane) for d in pair]
    return np.array([d.index for d in pair]).T        

get_node_pairs(pair = [front, back] , normal = "z").shape'''
0

0

In [45]:
main_template = """
********************************************************************************
**DISTRIBUTED MECHANICAL PROPERTIES ********************************************
********************************************************************************
**HEADER
*Preprint, echo=NO, model=NO, history=NO, contact=NO
********************************************************************************
** PART "pSAMPLE" DEFINITION
*Part, name = pSample
$MESH
********************************************************************************
**BOUNDARY CONDITIONS **********************************************************
********************************************************************************
$BOUNDARY_CONDITIONS
*End part
********************************************************************************
** ASSEMBLY ********************************************************************
********************************************************************************
*Assembly, name = Assembly
*Instance, name=iSample, part=pSample
*End Instance
*End Assembly
********************************************************************************
** MATERIALS *******************************************************************
********************************************************************************
$MATERIALS
********************************************************************************
** STEPS ***********************************************************************
********************************************************************************
**----------------------------------
** STEPS
*Step, Name=LOADING, Nlgeom=YES, Inc=1000000
*Static
0.01, 1, 1e-08, 0.01
** BOUNDARY CONDITIONS
*Boundary, op = NEW
iSample.control, 2, 2, 0.1
iSample.pinned, 1, 3
** LOADS
** RESTART OPTIONS 
*Restart, write, frequency=0
** FIELD OUTPUTS
*Output, field, frequency=1
*Node Output
U
*Element Output, directions=YES
E, PE, EE, PEEQ, S
** HYSTORY OUTPUTS
*Output, history
*Energy Output
ALLPD, ALLSE, ALLWK
*Node Output, nset=iSample.control
RF2
*Node Output, nset=iSample.control
CF2
*Node Output, nset=iSample.control
U2
*Element Output, elset=iSample.all, directions=NO
EVOL
*End Step
"""


In [64]:
%%time
if True:
    shape = np.array([10,10,10])
    Ne = shape.prod()
    E = 1.
    nu = .3
    dist = compmod2.distributions.Weibull(k = 5., l = 1.e-2)
    xt, x = dist.discretize(Ne, xmax = 1.)
    np.random.shuffle(x)
    materials = [argiope.materials.ElasticPerfectlyPlastic(
                                     label = "mat{0}".format(i+1), 
                                     young_modulus = E, 
                                     poisson_ratio = nu, 
                                     yield_stress = x * E) 
                                     for i in range(Ne)]
    
    sample = CuboidSample(shape = shape)
    sample.make_mesh()
    m = sample.mesh
    n = m.nodes
    c = n.coords
    xm, ym, zm = c.max(axis = 0)
    
    # MATERIALS & ELEMENTS
    m.elements.materials = ["mat{0}".format(i) for i in m.elements.index]
    m.elements.loc[:, ("type", "solver", "") ] = "C3D8"
    
    """
    # GENERAL PURPOSE ORDERED SETS
    n[("sets", "left")]      = (c.x == 0.) 
    n[("sets", "right")]     = (c.x == xm) 
    n[("sets", "bottom")]    = (c.y == 0.) 
    n[("sets", "top")]       = (c.y == ym) 
    n[("sets", "back")]      = (c.z == 0.) 
    n[("sets", "front")]     = (c.z == zm) 
    n[("sets", "pinned")]    = n.sets.left & n.sets.bottom & n.sets.back  
    
    control_node = n.index.max() + 1
    m.nodes.loc[control_node, [("coords")]] = xm * 1.1, ym * 1.1, zm * 1.1
    m.nodes.loc[control_node,"sets"] = False
    m.nodes[("sets", "control")] = False
    m.nodes.loc[control_node, ("sets", "control")] = True
    
    
    
    # BOUNDARY CONDITIONS
    bc_type = "periodic" # TEST
    if bc_type == "periodic":
        # FACE TO FACE PAIR EQUATIONS
        faces = {"uleft"   : n.coords[n.sets.left  ].sort_values(["y", "z"]),
                 "uright"  : n.coords[n.sets.right ].sort_values(["y", "z"]),    
                 "ubottom" : n.coords[n.sets.bottom 
                         & (n.sets.right == False)  ].sort_values(["x", "z"]),
                 "utop"    : n.coords[n.sets.top   
                         & (n.sets.right == False)  ].sort_values(["x", "z"]),
                 "uback"   : n.coords[n.sets.back  
                         & (n.sets.right == False)
                         & (n.sets.top == False)].sort_values(["x", "y"]),                   
                 "ufront"  : n.coords[n.sets.front 
                         & (n.sets.right == False)
                         & (n.sets.top == False)].sort_values(["x", "y"])}
        bc = "\n".join([argiope.utils._unsorted_set(v, k) 
                        for k, v in faces.items()]) + "\n"
    
        bc += argiope.utils._equation(nodes = ["uright", "uleft"], 
                                      dofs = [2,2], 
                                      coefficients = [1., -1.]) + "\n"
        bc += argiope.utils._equation(nodes = ["uright", "uleft"], 
                                      dofs = [3,3], 
                                      coefficients = [1., -1.]) + "\n"
        bc += argiope.utils._equation(nodes = ["utop", "ubottom"], 
                                      dofs = [1,1], 
                                      coefficients = [1., -1.]) + "\n"
        bc += argiope.utils._equation(nodes = ["utop", "ubottom"], 
                                      dofs = [3,3], 
                                      coefficients = [1., -1.]) + "\n"
        bc += argiope.utils._equation(nodes = ["ufront", "uback"], 
                                      dofs = [1,1], 
                                      coefficients = [1., -1.]) + "\n"
        bc += argiope.utils._equation(nodes = ["ufront", "uback"], 
                                      dofs = [2,2], 
                                      coefficients = [1., -1.]) + "\n"
        # NORMAL DISPLACEMENT
        bc += argiope.utils._equation(nodes = ["uright", "uleft", "control"], 
                                      dofs = [1,1,1], 
                                      coefficients = [1., -1., -1.],
                                      comment = "LEFT-RIGHT NORMAL PAIRS") + "\n"
        bc += argiope.utils._equation(nodes = ["utop", "ubottom", "control"], 
                                      dofs = [2,2,2], 
                                      coefficients = [1., -1., -1.],
                                      comment = "BOTTOM-TOP NORMAL PAIRS") + "\n"
        bc += argiope.utils._equation(nodes = ["ufront", "uback", "control"], 
                                      dofs = [3,3,3], 
                                      coefficients = [1., -1., -1.],
                                      comment = "BACK-FRONT NORMAL PAIRS") + "\n"

    """
    """
    out = Template(main_template).substitute(
        MESH = "", #sample.mesh.write_inp(),
        MATERIALS = "\n".join([mat.write_inp() for mat in materials]),
        BOUNDARY_CONDITIONS = bc.strip("\n"),
        )

    
    open("workdir/cuboid.inp", "w").write(out)
    """


    

CPU times: user 320 ms, sys: 0 ns, total: 320 ms
Wall time: 322 ms


In [55]:
%%time
data = {"MESH": sample.mesh.write_inp(),
        "MATERIALS": "", #"\n".join([mat.write_inp() for mat in materials]),
        "BOUNDARY_CONDITIONS": bc.strip("\n")}

#out = Template(main_template)

CPU times: user 3.78 s, sys: 0 ns, total: 3.78 s
Wall time: 3.78 s


In [69]:
%time s = m.write_inp()

CPU times: user 3.84 s, sys: 8 ms, total: 3.85 s
Wall time: 3.88 s


In [66]:
mesh = argiope.mesh.structured_mesh(shape = (10, 10, 10))
mesh.nodes[("sets", "truc")] = mesh.nodes.coords.x == 0.
%timeit mesh.write_inp()

48.3 ms ± 425 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [68]:
sample.mesh

(<Mesh, 1331 nodes, 1000 elements, 0 fields>,
 <Mesh, 1331 nodes, 1000 elements, 0 fields>)

In [71]:
m.elements

conn                                           materials  sets  \
           n0    n1    n2    n3    n4    n5    n6    n7             all   
                                                                          
element                                                                   
1           1     2    13    12   122   123   134   133      mat1  True   
2           2     3    14    13   123   124   135   134      mat2  True   
3           3     4    15    14   124   125   136   135      mat3  True   
4           4     5    16    15   125   126   137   136      mat4  True   
5           5     6    17    16   126   127   138   137      mat5  True   
6           6     7    18    17   127   128   139   138      mat6  True   
7           7     8    19    18   128   129   140   139      mat7  True   
8           8     9    20    19   129   130   141   140      mat8  True   
9           9    10    21    20   130   131   142   141      mat9  True   
10         10    11    22    21   131   132   143   142     mat10  True   
11         12    13    24    23   133   134   145   144     mat11  True   
12         13    14    25    24   134   135   146   145     mat12  True   
13         14    15    26    25   135   136   147   146     mat13  True   
14         15    16    27    26   136   137   148   147     mat14  True   
15         16    17    28    27   137   138   149   148     mat15  True   
16         17    18    29    28   138   139   150   149     mat16  True   
17         18    19    30    29   139   140   151   150     mat17  True   
18         19    20    31    30   140   141   152   151     mat18  True   
19         20    21    32    31   141   142   153   152     mat19  True   
20         21    22    33    32   142   143   154   153     mat20  True   
21         23    24    35    34   144   145   156   155     mat21  True   
22         24    25    36    35   145   146   157   156     mat22  True   
23         25    26    37    36   146   147   158   157     mat23  True   
24         26    27    38    37   147   148   159   158     mat24  True   
25         27    28    39    38   148   149   160   159     mat25  True   
26         28    29    40    39   149   150   161   160     mat26  True   
27         29    30    41    40   150   151   162   161     mat27  True   
28         30    31    42    41   151   152   163   162     mat28  True   
29         31    32    43    42   152   153   164   163     mat29  True   
30         32    33    44    43   153   154   165   164     mat30  True   
...       ...   ...   ...   ...   ...   ...   ...   ...       ...   ...   
971      1167  1168  1179  1178  1288  1289  1300  1299    mat971  True   
972      1168  1169  1180  1179  1289  1290  1301  1300    mat972  True   
973      1169  1170  1181  1180  1290  1291  1302  1301    mat973  True   
974      1170  1171  1182  1181  1291  1292  1303  1302    mat974  True   
975      1171  1172  1183  1182  1292  1293  1304  1303    mat975  True   
976      1172  1173  1184  1183  1293  1294  1305  1304    mat976  True   
977      1173  1174  1185  1184  1294  1295  1306  1305    mat977  True   
978      1174  1175  1186  1185  1295  1296  1307  1306    mat978  True   
979      1175  1176  1187  1186  1296  1297  1308  1307    mat979  True   
980      1176  1177  1188  1187  1297  1298  1309  1308    mat980  True   
981      1178  1179  1190  1189  1299  1300  1311  1310    mat981  True   
982      1179  1180  1191  1190  1300  1301  1312  1311    mat982  True   
983      1180  1181  1192  1191  1301  1302  1313  1312    mat983  True   
984      1181  1182  1193  1192  1302  1303  1314  1313    mat984  True   
985      1182  1183  1194  1193  1303  1304  1315  1314    mat985  True   
986      1183  1184  1195  1194  1304  1305  1316  1315    mat986  True   
987      1184  1185  1196  1195  1305  1306  1317  1316    mat987  True   
988      1185  1186  1197  1196  1306  1307  1318  1317    mat988  True   
989      1186  1187  1198  1197  1

In [ ]:
import argiope.utils


In [ ]:
def list_to_string(l = range(200), width = 40, indent = "  "):
    """
    Converts a list-like to string with given line width.
    """
    l = [str(v) + "," for v in l]
    counter = 0
    out = "" + indent
    for w in l:
        s = len(w)
        if counter + s > width: 
            out += "\n" + indent
            counter = 0
        out += w
        counter += s
    return out.strip(",")


print(list_to_string())

In [ ]:
argiope.utils.list_to_string([2])

In [ ]:
m.nodes.loc[1000, [("coords")]] = 10,10,10
m.nodes.loc[1000,"sets"] = False
m.nodes[("sets", "controlled")] = False

In [ ]:

n.coords[n.sets.left]


In [ ]:
n.coords[n.sets.left  ].sort_values(["y", "z"],ascending = False)

In [ ]:
n.coords[n.sets.right].sort_values(["y", "z"], ascending = False)

In [ ]:
dist = compmod2.distributions.Weibull(k = 5., l = .01)
xt, x = dist.discretize(Ne, xmax = 1. )

In [ ]:
import compmod2.distributions

In [30]:
eq = argiope.utils._equation
%timeit eq(nodes = ["uright", "uleft", "control"], dofs = [1,1,1], coefficients = [1., -1., -1.])


8.81 µs ± 366 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [38]:
%timeit s = sample.mesh.write_inp()

3.56 s ± 35.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
mesh = argiope.mesh.structured_mesh(shape = (10, 10, 10))
mesh.nodes[("sets", "truc")] = mesh.nodes.coords.x == 0.
%timeit mesh.write_inp()

49.3 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
